## Thank you for visit


#### In this notebook, I want to show Example how to calculate MAP@12 

#### If you like this notebook, upvote please 😉

In [1]:
import numpy as np
import pandas as pd
import gc
import os
import time
import random
from tqdm.auto import tqdm

In [2]:
def visualize_df(df):
    print(df.shape)
    display(df.head())

In [3]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
visualize_df(transactions_train)

(31788324, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [4]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
del sub['prediction']; gc.collect()
visualize_df(sub)

(1371980, 1)


,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [5]:
# transactions_train['t_dat'].unique()[-7:]

# array(['2020-09-16', '2020-09-17', '2020-09-18', '2020-09-19',
#       '2020-09-20', '2020-09-21', '2020-09-22'], dtype=object)

In [6]:
val_start_date = '2020-09-16'

train_data = transactions_train.query(f"t_dat < '{val_start_date}'").reset_index(drop=True)
valid_data = transactions_train.query(f"t_dat >= '{val_start_date}'").reset_index(drop=True)

visualize_df(train_data)
visualize_df(valid_data)

(31548013, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


(240311, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,786022008,0.048441,2
1,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,913272003,0.032288,2
2,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,889669006,0.056508,2
3,2020-09-16,0010e8eb18f131e724d6997909af0808adbba057529edb...,237347060,0.033881,1
4,2020-09-16,0010e8eb18f131e724d6997909af0808adbba057529edb...,562245001,0.013542,1


In [7]:
train_unq = train_data.groupby('customer_id')['article_id'].apply(list).reset_index()
train_unq['valid_pred'] = train_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
visualize_df(train_unq)

(1356709, 3)


,customer_id,article_id,valid_pred
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[625548001, 176209023, 627759010, 697138006, 5...",0625548001 0176209023 0627759010 0697138006 05...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[583558001, 639677008, 640244003, 521269001, 6...",0583558001 0639677008 0640244003 0521269001 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[663713001, 541518023, 663713001, 578020002, 7...",0663713001 0541518023 0663713001 0578020002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[742079001, 732413001]",0742079001 0732413001
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[634249005, 677049001, 698286003, 707704003, 3...",0634249005 0677049001 0698286003 0707704003 03...


In [8]:
valid_unq = valid_data.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_unq['valid_true'] = valid_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
visualize_df(valid_unq)

(68984, 3)


,customer_id,article_id,valid_true
0,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,[624486001],0624486001
1,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,[827487003],0827487003
2,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,"[757926001, 788575004, 640021019]",0757926001 0788575004 0640021019
3,000525e3fe01600d717da8423643a8303390a055c578ed...,[874110016],0874110016
4,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,"[903762001, 879189005, 158340001, 867966009, 9...",0903762001 0879189005 0158340001 0867966009 09...


In [9]:
merged = pd.merge(sub, train_unq, on='customer_id', how='left').fillna('')
merged = pd.merge(merged, valid_unq, on='customer_id', how='left').fillna('')

del merged['article_id_x'], merged['article_id_y']; gc.collect()
merged.head()

,customer_id,valid_pred,valid_true
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0625548001 0176209023 0627759010 0697138006 05...,
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0583558001 0639677008 0640244003 0521269001 06...,
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001 0541518023 0663713001 0578020002 07...,
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001,
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0634249005 0677049001 0698286003 0707704003 03...,


In [10]:
def precision_k(row) -> float:
    actual = row['valid_true'].split() # prediction_string --> prediction list
    predicted = row['valid_pred'].split() # prediction_string --> prediction list
    k = 12

    actual_set = set(actual)
    predicted_set = set(predicted[-k:])
    
    precision_k_value = len(actual_set & predicted_set)/k
    
    return precision_k_value

In [11]:
tqdm.pandas()

merged['Precision@12'] = merged.progress_apply(precision_k, axis=1)
merged['Precision@12'].head()

  0%|          | 0/1371980 [00:00<?, ?it/s]

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Precision@12, dtype: float64

In [12]:
# scored over 0 data

merged[merged['Precision@12']>0]

,customer_id,valid_pred,valid_true,Precision@12
330,000fb6e772c5d0023892065e659963da90b1866035558e...,0674482003 0688873001 0670063001 0403448007 04...,0786022008 0913272003 0889669006 0913272003 08...,0.083333
647,001ddeb8fb74fec5693116da83b488e05ee9a9e179f3fd...,0682965001 0539197011 0539197011 0649098002 06...,0706016002 0876148002 0843777005 0812271001,0.083333
687,00201f34c8c92683263346d78c2b45ffd0c6927229542d...,0551515002 0481696002 0481696002 0651692001 06...,0903926001 0909320002,0.083333
842,00282135561702f5b3b750fa3382d8fd83ce5d761a507e...,0573626001 0688632001 0569974001 0570002001 05...,0855249006 0822171001,0.083333
1108,0034669aa967dd26cb88d0f20d99d381741a50e44d3be5...,0696953001 0551044015 0936862001,0936862001 0936862001 0936862001,0.083333
...,...,...,...,...
1370190,ffa938ef7d86f1d9793605bc3fc2a6736fb44b902984df...,0656649009 0789331003 0704128001 0790057001 07...,0611415034 0611415034 0805000008 0862271003 08...,0.083333
1371091,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,0507909022 0507909004 0507909004 0633130002 06...,0896169005 0791587021 0791587021 0910949002 07...,0.083333
1371341,ffe1791c9c6e3df9aafeebc77cf2cf03dd0123ac76ef90...,0501722012 0667770001 0454544010 0667770002 04...,0853740002 0853740002 0853740017 0853740017 08...,0.083333
1371602,ffee12091eb3aa2337defe250d7f6b23dbbf9d496fdd7c...,0616733001 0368979001 0693748001 0647463001 06...,0902528007 0902528007 0805947006,0.083333


In [13]:
# mean average precision at 12

round(merged['Precision@12'].mean(), 5)

0.00036

#### Simple last ~12 purchased as predictions scored 0.00036

#### We have only the third decimal place on LB

#### This sub maybe scored 0.000 😂

In [14]:
sub = merged[['customer_id', 'valid_pred']].copy()
sub.columns = ['customer_id', 'prediction']
print(sub.shape)

sub.to_csv('submission.csv', index=False)

(1371980, 2)
